In [1]:
import networkx as nx
import dwave_networkx as dnx
import matplotlib.pyplot as plt
import random
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
from pyqubo import Spin
import neal
import time
import numpy as np
import pandas as pd
from greedy import SteepestDescentSolver

### Real SAT problem instance in architecture

In [62]:
def ising_and_embedding_real(a, b): #Assuming we have a SAT problem all positive (no negations)
    #List of all the qubits and couples of Advantage_system4.1 (Pegasus)
    couplers_indices = DWaveSampler().edgelist
    qubit_indices = DWaveSampler().nodelist
    
    #We start only for qubits 30-50
    couplers=[]
    qubits=[]
    h={}
    J={}
    SAT=""
    for couple in couplers_indices:
        if a <= couple[0] <= b:
            couplers.append(couple)
            if couple[0] not in qubits:
                qubits.append(couple[0])
                v0=random.choice([0.25,-0.25])
                h[couple[0]] = v0
                SAT += str(couple[0])+" " if v0 < 0 else str(-couple[0])+" "
            else:
                v0=random.choice([0.25,-0.25])
                h[couple[0]] += v0
                SAT += str(couple[0])+" " if v0 < 0 else str(-couple[0])+" "
            if couple[1] not in qubits:
                qubits.append(couple[1])
                v1=random.choice([0.25,-0.25])
                h[couple[1]] = v1
                SAT += str(couple[1])+" " if v1 < 0 else str(-couple[1])+" "
            else:
                v1=random.choice([0.25,-0.25])
                h[couple[1]] += v1
                SAT += str(couple[1])+" " if v1 < 0 else str(-couple[1])+" "
            SAT += "\n"
            J[couple[0],couple[1]] = 0.25 if v0>0 or v1>0 else -0.25  #There will only be one couple per coupling_factor existing, e.q. no clause is repeated
    
    #Define embedding
    embedding = {q: [q] for q in qubits} #Because the nodelist and the edgelist are from the architecture, we can assign each node/variable to a qubit on the embedding

    return h, J, embedding, SAT

In [63]:
h_r, J_r, embedding_r, SAT = ising_and_embedding_real(300,1300) #The values are the range of the "central" qubits we get (from each of these qubits we will also get its neighbours).
                                             #It is important to know that the first qubit in the architecture is the number 30

In [64]:
num_reads=100
anneal_time=100.0
secure_q = input("Do you really want to use the QPU? (y/n)")
if secure_q == 'y' or secure_q == 'Y':
    sampler = FixedEmbeddingComposite(DWaveSampler(token='DEV-291d80af600d6eb433a8019c579070ba37436e9a'), embedding=embedding_r)
    sample_set_real = sampler.sample_ising(h_r, J_r, num_reads=num_reads, annealing_time=anneal_time, return_embedding=True)
    print("Using DWaveSampler()")
    print(sample_set_real)

Do you really want to use the QPU? (y/n) y


Using DWaveSampler()
   300 301 302 303 304 305 306 307 308 309 310 ... 5720   energy num_oc. ...
0   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2162.25       1 ...
1   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2162.25       1 ...
2   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2161.25       1 ...
3   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2161.25       1 ...
4   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
5   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
6   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
7   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
8   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
9   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
10  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1 -2160.25       1 ...
11  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1 ...   +1

In [73]:
qpu_access_time = sample_set_real.info['timing']['qpu_access_time']
#Tiempo usado por la QPU para resolver el problema de forma activa
qpu_sampling_time = sample_set_real.info['timing']['qpu_sampling_time']
print("Using DWave QPU... QPU access time:", qpu_access_time*10**(-6), "s.", " ; QPU sampling time:", qpu_sampling_time*10**(-6), "s.")

Using DWave QPU... QPU access time: 0.04920317 s.  ; QPU sampling time: 0.033431999999999996 s.


In [65]:
dwave.inspector.show(sample_set_real)

Serving Inspector on http://127.0.0.1:18000/?problemId=a5a91b94-c6d8-4250-b5a0-a76212ef37c4

'http://127.0.0.1:18000/?problemId=a5a91b94-c6d8-4250-b5a0-a76212ef37c4'

##### Solving with Simulated Annealing

In [66]:
ising_model = dimod.BinaryQuadraticModel(h_r, J_r, 0.0, dimod.Vartype.SPIN)
sampler = neal.SimulatedAnnealingSampler()
start_time = time.time()
response_real = sampler.sample(ising_model, num_reads=100)
end_time = time.time()

print("Using SimulatedAnnlearingSampler()... Time:", round(end_time-start_time, 2), "s.")
print(response_real)

Using SimulatedAnnlearingSampler()... Time: 5.35 s.
   300 301 302 303 304 305 306 307 308 309 310 311 ... 5720   energy num_oc.
26  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2163.25       1
52  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2163.25       1
80  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2163.25       1
5   -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
15  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
16  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
18  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
19  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
20  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
21  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
22  -1  +1  +1  -1  +1  +1  +1  -1  +1  -1  +1  +1 ...   +1 -2162.25       1
24  -1  +1  +1  -1  +1  

## Lets study if the results are close to the exacto solution

In [67]:
SAT

'300 301 \n-300 -315 \n-300 2971 \n-300 -2986 \n-300 -3000 \n-300 -3015 \n300 3030 \n-300 3045 \n-300 3061 \n-300 3076 \n300 3091 \n300 -3106 \n300 3121 \n-300 3136 \n301 -302 \n301 316 \n-301 -3151 \n-301 3166 \n-301 3180 \n-301 -3195 \n-301 -3210 \n301 3225 \n301 3241 \n301 -3256 \n-301 -3271 \n-301 3286 \n-301 -3301 \n301 3316 \n-302 -303 \n302 -317 \n-302 3331 \n302 3346 \n-302 3360 \n302 -3375 \n302 3390 \n302 3405 \n-302 -3421 \n-302 3436 \n302 3451 \n302 -3466 \n302 -3481 \n302 3496 \n303 -304 \n303 -318 \n303 -3511 \n-303 -3526 \n303 3540 \n303 3555 \n-303 3570 \n-303 3585 \n303 3601 \n-303 3616 \n-303 -3631 \n-303 3646 \n-303 -3661 \n-303 3676 \n304 305 \n304 319 \n-304 3691 \n304 3706 \n-304 -3720 \n-304 -3735 \n-304 -3750 \n-304 3765 \n304 3781 \n-304 3796 \n304 -3811 \n304 -3826 \n304 -3841 \n304 3856 \n305 -306 \n305 320 \n-305 -3871 \n-305 3886 \n305 3900 \n-305 -3915 \n-305 -3930 \n-305 3945 \n305 -3961 \n-305 -3976 \n-305 -3991 \n-305 4006 \n305 -4021 \n-305 4036 \n306 

In [68]:
#From lines to clauses
def parsear_cnf(sat):
    lineas = sat.split("\n")
    clausulas = [list(map(int, lineas.split())) for lineas in lineas]

    return clausulas

clauses = parsear_cnf(SAT)
print("El número total de cláusulas es: ", len(clauses))

El número total de cláusulas es:  13390


In [69]:
def count_unsatisfied_clauses(assignment, clauses):
    count = 0
    for clause in clauses:
        for literal in clause:
            if (literal > 0 and assignment[literal] == 1) or (literal < 0 and assignment[-literal] == -1):
                count += 1
                break
    return str(len(clauses)-count)

unsat = count_unsatisfied_clauses(sample_set_real.first.sample, clauses)
unsat_sim = count_unsatisfied_clauses(response_real.first.sample, clauses)

print("El número de unsatisfied clauses para la solución de D'Wave es: ", unsat)
print("Para el Simulated Annealing es: ", unsat_sim)

El número de unsatisfied clauses para la solución de D'Wave es:  2847
Para el Simulated Annealing es:  2820


#### Testing with Max2SAT exact solvers

In [74]:
#TEST para un Max2SAT solver


from pysat.solvers import Glucose3

max2sat_formula = [[1, 2], [-1, 3], [-2, 4], [-3, -4]]

solver = Glucose3()

for clause in max2sat_formula:
    solver.add_clause(clause)

result = solver.solve()

if result:
    assignment = solver.get_model()
    #unsatisfied_clauses = solver.not_conflicts()
    print("Satisfying assignment:", assignment)
    #print("Number of unsatisfied clauses:", unsatisfied_clauses)
else:
    print("No satisfying assignment found.")

Satisfying assignment: [1, -2, 3, -4]
